In [2]:
using Agents
using Random
using CairoMakie # for clips

In [6]:
@agent struct Crook(GridAgent{5})
    captured::Int64
    velocity:: Float64
    position:: Tuple{Int64}
    weapon::Bool
    weapontype:: Vector #[1,2,3] (knife, gun, rope)
end

@agent struct Cop(GridAgent{4})
    k9::Bool
    velocity:: Float64
    position:: Tuple{Int64}
    k9level:: Vector 
end

In [7]:
space = GridSpaceSingle((100,100, 10); periodic = false)

GridSpaceSingle with size (100, 100, 10), metric=chebyshev, periodic=false

In [8]:
function create_battlefield(; Crook = 50, seed = 6547)
    model = StandardABM(
        Crook,
        GridSpace((100, 100, 10); periodic = false);
        agent_step!,
        scheduler = Schedulers.Randomly(),
        rng = Random.Xoshiro(seed),
    )

    n = 0
    while n != fighters
        pos = (rand(abmrng(model), 1:100, 2)..., 1) # Start at level 1
        if isempty(pos, model)
            add_agent!(pos, model, false, 0, :diamond)
            n += 1
        end
    end

    return model
end

create_battlefield (generic function with 1 method)